In [1]:
import os
import sys
import platform

import torch
import torch.nn
import torchvision.datasets


from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.utils import save_image

In [2]:
print("Python Version: {}".format(platform.python_version()))
print("Torch Version: {}".format(torch.__version__))

Python Version: 3.7.1
Torch Version: 0.4.1


# Get MNIST Data

In [3]:
num_epochs = 11
batch_size = 128
learning_rate = 1e-3

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

In [5]:
mnistTrainData = torchvision.datasets.MNIST("Data", download=True, train=True, transform=transform)
mnistTestData = torchvision.datasets.MNIST("Data", download=True, train=False, transform=transform)

In [6]:
print("Number of Training Samples: {}".format(len(mnistTrainData)))
print("Number of Testing Samples: {}".format(len(mnistTestData)))

Number of Training Samples: 60000
Number of Testing Samples: 10000


In [7]:
trainDataLoader = torch.utils.data.DataLoader(mnistTrainData, batch_size=batch_size, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(mnistTestData, batch_size=batch_size, shuffle=True)

# Autoencoder Model

In [8]:
class Autoencoder(torch.nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = torch.nn.Sequential(torch.nn.Linear(28 * 28, 20*20))
        self.decoder = torch.nn.Sequential(torch.nn.Linear(20*20, 28 * 28), torch.nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Train Model

In [9]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [10]:
model = Autoencoder()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(trainDataLoader):
        img = data.view(data.size(0), -1)
        img = Variable(img)
        
        output = model(img)
        loss = criterion(output, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, 'image_{}.png'.format(epoch))


epoch [1/11], loss:0.0154
epoch [2/11], loss:0.0085
epoch [3/11], loss:0.0063
epoch [4/11], loss:0.0052
epoch [5/11], loss:0.0049
epoch [6/11], loss:0.0048
epoch [7/11], loss:0.0046
epoch [8/11], loss:0.0041
epoch [9/11], loss:0.0045
epoch [10/11], loss:0.0042
epoch [11/11], loss:0.0045


# Visualize Ouput

In [14]:
mnistTrainData[1][1]

tensor(0)

In [15]:
x = torch.nn.Tanh()(model.encoder(mnistTrainData[1][0].view(-1)))

In [16]:
x = 0.5 * (x + 1)
x = x.clamp(0, 1)
x = x.view(1, 1, 20, 20)
save_image(x, 'test.png'.format(epoch))

In [19]:
x = model(mnistTrainData[1][0].view(-1))

In [20]:
x = 0.5 * (x + 1)
x = x.clamp(0, 1)
x = x.view(1, 1, 20, 20)
save_image(x, 'test2.png'.format(epoch))

RuntimeError: invalid argument 2: size '[1 x 1 x 20 x 20]' is invalid for input with 784 elements at /Users/soumith/miniconda2/conda-bld/pytorch_1532624435833/work/aten/src/TH/THStorage.cpp:84

# Visualize Latent Properties